In [ ]:
!pip install googlemaps

In [ ]:
import pandas as pd
pd.set_option('max_columns', 100)
import numpy as np
from collections import Counter
import re
import datetime as dt
import googlemaps
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("gmaps")
import os
import glob
import tqdm
import random
import warnings
warnings.filterwarnings('ignore')
import pickle
import base64
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import HTML

# NFL Big Data Bowl 2021 
The **National Football League (NFL)** is a professional American football league consisting of 32 teams, divided equally between the National Football Conference (**NFC**) and the American Football Conference (**AFC**). The NFL is one of the four major North American professional sports leagues, the highest professional level of American football in the world, the wealthiest professional sport league by revenue, and the sport league with the most valuable teams.The NFL's 17-week regular season runs from early September to late December, with each team playing 16 games and having one bye week. Following the conclusion of the regular season, seven teams from each conference (four division winners and three wild card teams) advance to the playoffs, a single-elimination tournament culminating in the Super Bowl, which is usually held on the first Sunday in February and is played between the champions of the NFC and AFC.
<br>
<img src='https://images.daznservices.com/di/library/sporting_news/65/c8/lambeau-field-080515-getty-ftrjpg_17qhlqkbvtfti1t9g9rx5mhyku.jpg?t=-20443826&quality=100' height=500 width=700/>
<br>
With the help of this notebook, I'll be doing an extensive study of the **2018 NFL Seaso**n. Along with that, the data bowl also provides us with all the player tracking data for all drop-back pass plays from the 2018 regular season so that we can measure defensive performance on these plays.

> Here's a one minute introductory video for all the beginners out there like me 😜

In [ ]:
display(HTML('<iframe width="700" height="500" src="https://www.youtube.com/embed/3t6hM5tRlfA" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>'))

In [ ]:
games = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2021/games.csv')
players = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2021/players.csv')
plays = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2021/plays.csv')

# 🏈 NFL Season Schedule
The NFL's 17-week regular season runs from early September to late December, with each team playing 16 games and having one bye week. 

The below plot shows the game start timings and their respective dates on which they were played. Hovering on the points will provide further details about the game.

In [ ]:
def add_image_plotly(fig,filename,**kwargs):
    """Adds background image to plotly figure."""
    img = base64.b64encode(open(filename, 'rb').read())
    fig.update_layout(images=[
        dict(
            source  = 'data:image/png;base64,{}'.format(img.decode()),
            xref    = kwargs.get('xref','paper'), 
            yref    = kwargs.get('yref','paper'),
            x       = kwargs.get('x',0),
            y       = kwargs.get('y',1),
            sizex   = kwargs.get('sizex',0.5),
            sizey   = kwargs.get('sizey',0.5),
            xanchor = kwargs.get('xanxhor','left'),
            yanchor = kwargs.get('yanchor','top'),
            opacity = kwargs.get('opacity',0.5),
            sizing  = kwargs.get('sizing',None),
            layer   = kwargs.get('layer','below')
        )
    ])

In [ ]:
games['gameDateT'] = pd.to_datetime(games.gameDate, format='%m/%d/%Y')
games['hours'] = games.gameTimeEastern.apply(lambda x: \
                                          np.round(sum([int(num)/(60**i) \
                                          for i,num in enumerate(x.split(':'))]),2))
games['game'] = games.apply(lambda x: f"{x.homeTeamAbbr} vs {x.visitorTeamAbbr}",axis=1)

In [ ]:
fig = px.scatter(data_frame=games, x='gameDateT', y='hours', color='week', hover_name='game', custom_data=['gameTimeEastern'])
add_image_plotly(fig,'/kaggle/input/nfl2021images/NFL.png',x=0.4,y=0.8,sizex=1,sizey=0.6)
fig.update_traces(hovertemplate = '<b>%{hovertext}</b><br><br>'+
                  '<b>Game Date</b>: %{x}<br><b>Time</b>: %{customdata[0]}<br>'+
                  '<b>Week</b>: %{marker.color}<extra></extra>')
fig.update_layout(
    template="plotly_dark",title = 'NFL Schedule, 2018 Season', title_font_family='Rockwell',
    width=700,height=300,
    xaxis=dict(title='Game Date'),
    yaxis=dict(title='Hour of the Day'))
fig.show()

# 🏈 NFL Players
There are so many ways to break down how NFL teams build their roster.  Some teams build with youth while others collect veteran players.  While no one formula for stacking an NFL roster is iron-clad, one rule in the NFL is: Teams are only allowed to have 53 players on their active roster.  Of these 53, only 46 players can dress out for the actual game. 
<img src='https://i.pinimg.com/originals/85/54/6e/85546e169ea765819ec6ae68fb0f66a1.png'/>

In American football, the specific role that a player takes on the field is referred to as their **position**. Under the modern rules of American football, both teams are allowed 11 players on the field at one time and have **unlimited free substitutions**, meaning that they may change any number of players during any dead ball situation. This has resulted in the development of three task-specific **platoons** of players within any single team: the `offense` (the team with possession of the ball, which is trying to score), the `defense` (the team trying to prevent the other team from scoring, and to take the ball from them), and the so-called `special teams` (who play in all kicking situations). Within these three separate platoons, various positions exist depending on the job that player is doing.

In [ ]:
d_def = {
    'CB': 'Cornerbacks',
    'SS': 'Safeties',  #'Strong Safeties',
    'FS': 'Safeties',  #'Free Safeties',
    'MLB': 'Linebackers',  #'Middle Linebackers',
    'ILB': 'Linebackers', #'Inside Linebackers',
    'OLB': 'Linebackers', #'Outside Linebackers',
    'DE': 'Defensive ends',
    'LB': 'Linebackers',
    'DB': 'Defensive Backs',
    'S': 'Safeties',
    'NT': 'Nose Tackles',
    'DT': 'Defensive Tackles'
}

d_ofs = {
    'WR': 'Wide Recievers',
    'QB': 'Quarterbacks',
    'TE': 'Tight Ends',
    'RB': 'Running Backs',
    'FB': 'Fullbacks',
    'HB': 'Halfbacks',
    
}

d_spe = {
    'P': 'Punters',
    'K': 'Kickers',
    'LS': 'Long Snappers'
}
def generate_colors(n):
    colors = []
    random.seed(42)
    for i in range(n):
        rgb_color=f"rgb({random.randint(0,255)},{random.randint(0,255)},{random.randint(0,255)})"
        colors.append(rgb_color)
    return colors

In [ ]:
players.birthDate = pd.to_datetime(players.birthDate)
players['Age'] = dt.datetime.now() - players.birthDate
players.Age = np.round(players.Age.dt.days/365.25, 2)
players.height = players.height.apply(lambda x: int(x) if '-' not in x \
                     else round(int(x.split('-')[0])*12+int(x.split('-')[1])))
players['height_jitter'] = players.height.apply(lambda x: x + random.randrange(-50,51)/100)
players['weight_jitter'] = players.weight.apply(lambda x: x + random.randrange(-50,51)/100)
players['pos_desc'] = players.position.apply(lambda x: d_def.get(x,d_ofs.get(x,d_spe.get(x,None))))
players['type'] = players.position.apply(lambda x: 'Offensive' if x in d_ofs.keys() \
                                         else 'Defensive' if x in d_def.keys() \
                                         else 'Special')

# 🏈🏈 Player Age Distribution
The below plot shows the age distribution of the three task-specific platoons of players. 
- All the three platoons share quite a similar age distribution with Offense and Defence having more younger players (almost 75%) below the age of 30. 
- Offense and Defense have outliers too with one player reaching an age of 43 years. While, Special teams don't have any outliers. 
- The youngest player in NFL Season 2018 was a 22.52 year old defensive player while the eldest being a 43.26 years old offensive player.

In [ ]:
fig = px.box(data_frame=players, x='Age', color='type',template='plotly_dark',width=700, height=300)
                 # barmode='overlay',nbins=20)
fig.for_each_trace(lambda trace: trace.update(hovertemplate=trace.hovertemplate.replace('=',': ')))
fig.update_layout(title = 'NFL Player Age, 2018 Season', title_font_family='Rockwell',
                  legend=dict(title='Type', x=0.18,y=1.1,orientation='h'), xaxis_title='Age')
fig.show()

# 🏈🏈 Player Positions
Within the three platoons: Offense, Defence & Special teams, various positions exist depending on the job that player is doing. The figure shows these various positions. Its missing the special teams' positions but just like their name, they come into only in all kicking situations. So, they don't have a particular position.
<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/2/25/American_Football_Positions.svg/800px-American_Football_Positions.svg.png' height=400 width=500/>

Click on this [link](https://en.wikipedia.org/wiki/American_football_positions) to know all the positions in NFL. In this Dataset we are provided with the following positions:-

<h3>Offense</h3>
<ul style="list-style-type:square;">
  <li>Wide Recievers <span class="label label-danger">WR</span></li>
  <li>Quarterbacks <span class="label label-danger">QB</span></li>
  <li>Tight Ends <span class="label label-danger">TE</span></li>
  <li>Running Backs <span class="label label-danger">RB</span></li>
  <li>Fullbacks <span class="label label-danger">FB</span></li>
  <li>Halfbacks <span class="label label-danger">WR</span></li>
</ul>
<h3>Defence</h3>
<ul style="list-style-type:square;">
  <li>Cornerbacks <span class="label label-info">CB</span></li>
  <li>Safeties <span class="label label-info">S</span></li>
      <ul style="list-style-type:circle;">
          <li>Strong Safeties <span class="label label-info">SS</span></li>
          <li>Free Safeties <span class="label label-info">FS</span></li>
      </ul>
  <li>Linebackers <span class="label label-info">LB</span></li>
      <ul style="list-style-type:circle;">
          <li>Middle Linebackers <span class="label label-info">MLB</span></li>
          <li>Inside Linebackers <span class="label label-info">ILB</span></li>
          <li>Outside Linebackers <span class="label label-info">OLB</span></li>
      </ul>  
  <li>Defensive ends <span class="label label-info">DE</span></li>
  <li>Nose Tackles <span class="label label-info">NT</span></li>
  <li>Defensive Tackles <span class="label label-info">DT</span></li>
  <li>Halfbacks <span class="label label-info">WR</span></li>
</ul>
<h3>Special Teams</h3>
<ul style="list-style-type:square;">
  <li>Punters <span class="label label-success">P</span></li>
  <li>Kickers <span class="label label-success">K</span></li>
  <li>Long Snappers <span class="label label-success">LS</span></li>
</ul>

In [ ]:
if os.path.exists('/kaggle/input/nflcolleges/college.pickle'):
    with open('/kaggle/input/nflcolleges/college.pickle', 'rb') as handle:
        d = pickle.load(handle)
else:
    gmaps = googlemaps.Client(key=secret_value_0)
    d = {}
    colleges = players.collegeName.unique().tolist()
    for college in colleges:
        data = gmaps.geocode(college)
        d[college] = data[0]['geometry']['location']
    with open('college.pickle', 'wb') as handle:
        pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
def get_coords(college):
    lat = d[college]['lat']
    lng = d[college]['lng']
    return (lat,lng)

players['Latitude'] = players.apply(lambda x: get_coords(x['collegeName'])[0],axis=1)
players['Longitude'] = players.apply(lambda x: get_coords(x['collegeName'])[1],axis=1)
players['num_of_players'] = players.collegeName.map(Counter(players.collegeName))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scattergeo(
    lat=players['Latitude'],
    lon=players['Longitude'],
    marker_size=players['num_of_players'],
    text = players['collegeName'].astype(str) +\
           ' (' +players['num_of_players'].astype(str) +')',
    hoverinfo='text'
))
fig.update_layout(
        title = 'Players and their respective Colleges',
        template='plotly_dark',
        title_font_family='Rockwell',
        margin=dict(l=0,r=0,t=50,b=0),
        geo_scope='usa',
        width=700,
        height=300
    )
fig.show()

In [ ]:
fig = px.scatter(
    data_frame=players,
    x='height_jitter',
    y='weight_jitter',
    width=700, height=400,
    color='pos_desc',
    facet_col='type',
    trendline="ols",
    opacity=0.3, 
    custom_data=['displayName','Age','type','collegeName','position']
)

fig.update_traces(line_width=5)
hovertemplate='<b>%{customdata[0]}</b><br><br>'+\
              '<b>Position</b>: %{customdata[4]}<br>'+\
              '<b>Age</b>: %{customdata[1]} years<br>'+\
              '<b>Height</b>: %{x} inches<br><b>Weight</b>: %{y} lbs<br>'+\
              '<b>College</b>: %{customdata[3]}'
fig.for_each_trace(lambda trace: trace.update(hovertemplate=None)\
                   if 'OLS trendline' in trace.hovertemplate \
                   else trace.update(hovertemplate=hovertemplate))

for col in range(1,4):
    label = 'Height (inches)' if col == 2 else None 
    fig.update_xaxes(title_text=label,row=1, col=col), 
fig.update_yaxes(title_text='Weight (lbs)', row=1, col=1)

add_image_plotly(fig,'/kaggle/input/nfl2021images/Daco_6014349.png',x=-0.2,y=1,sizex=1,sizey=1)

fig.for_each_annotation(lambda annot : annot.update(text=annot.text[5:]))

fig.update_layout(
    title = 'NFL Player Size, 2018 Season',
    title_font_family='Rockwell',
    template='plotly_dark', 
    legend_title='Position',
    legend_font_size=8,
    legend_font_family='Rockwell'
)
fig.show()

In [ ]:
player_game_stats = pd.DataFrame()
files = glob.glob('/kaggle/input/nfl-big-data-bowl-2021/week*.csv')
for file in tqdm.tqdm(files):
    week_data = pd.read_csv(file)
    week_data = week_data[week_data['displayName']!='Football']
    week_data['jerseyNumber'] = week_data['jerseyNumber'].astype(np.int8)
    week_data['nflId'] = week_data['nflId'].astype(np.int64)
    player_game_stats = player_game_stats.append(week_data.groupby(['gameId','nflId'],
                        as_index=False).agg({'s':'mean','a':'mean','dis':'sum'}))

In [ ]:
player_stats = player_game_stats.groupby('nflId',as_index=False)\
               .agg(avg_speed = ('s',np.mean),avg_acc = ('a',np.mean),
                    avg_distance = ('dis',np.mean),total_distance = ('dis',np.sum))

player_stats = pd.merge(players, player_stats, on='nflId')

In [ ]:
columns = ['displayName','position','pos_desc','collegeName','height','weight',
           'avg_speed','avg_acc','avg_distance','total_distance']
display_cols = ['displayName','position','collegeName','height_feet','weight','avg_speed','avg_acc','total_distance']
player_stats_req = player_stats[columns]
for col in columns[-4:]:
    player_stats_req[col] = np.round(player_stats_req[col],2)
player_stats_req['height_feet'] = player_stats_req['height'].apply(lambda x: f"{int(x/12)}' {x%12}\"")
player_stats_req.sort_values('total_distance', ascending=False, inplace=True)

In [ ]:
MARKER_SIZE = 7
LINE_WIDTH = 0.5
fig = make_subplots(rows=3,cols=2,specs=[[{'type':'table','colspan':2},None],[{},{}],[{},{}]])
fig.add_trace(go.Table(
        columnorder=[1,2,3,4,5,6,7,8],
        columnwidth=[100,60,120,50,50,100,120,100],
        header=dict(
            values=["<b>Name</b>", "<b>Position</b>", "<b>College</b>",
                    "<b>Height</b><br>in feet",'<b>Weight</b><br>in lbs',
                    "<b>Avg. Speed</b><br>in yards/s",
                    "<b>Avg. Acceleration</b><br>in yards/s<sup>2</sup>",
                    "<b>Total Distance</b><br>covered in yards"],
            line_color='black',
            fill_color='black',
            font=dict(color='white', size=8),
            align=['center']
        ),
        cells=dict(
            values=[player_stats_req[k].tolist() for k in display_cols],
            align = "center",
            line_color='black',
            fill=dict(color=['paleturquoise', 'black']),
            font=dict(size=8,color=['black','white']))
    ),1,1)
positions = player_stats_req.pos_desc.unique().tolist()
colors = generate_colors(len(positions))
for i,position in enumerate(positions):
    df_plot = player_stats_req[player_stats_req['pos_desc']==position]
    fig.add_trace(go.Scatter(name=position,legendgroup=position,showlegend=True,
                         x=df_plot.weight,y=df_plot.avg_speed,mode='markers',opacity=0.7,
                             marker_color = colors[i],
                         #marker_size=MARKER_SIZE,marker_line_width=LINE_WIDTH,
                         text=player_stats_req['displayName'],
                         textposition='bottom center'),2,1)
    fig.add_trace(go.Scatter(name=position,legendgroup=position,showlegend=False,
                         x=df_plot.weight,y=df_plot.avg_acc,mode='markers',opacity=0.7,
                             marker_color = colors[i],
                         #marker_size=MARKER_SIZE,marker_line_width=LINE_WIDTH,
                         text=player_stats_req['displayName'],
                         textposition='bottom center'),2,2)
    fig.add_trace(go.Scatter(name=position,legendgroup=position,showlegend=False,
                         x=df_plot.avg_speed,y=df_plot.avg_acc,mode='markers',opacity=0.7,
                             marker_color = colors[i],
                         #marker_size=MARKER_SIZE,marker_line_width=LINE_WIDTH,
                         text=player_stats_req['displayName'],
                         textposition='bottom center'),3,1)
    fig.add_trace(go.Scatter(name=position,legendgroup=position,showlegend=False,
                         x=df_plot.height,y=df_plot.total_distance,mode='markers',opacity=0.7,
                             marker_color = colors[i],
                         #marker_size=MARKER_SIZE,marker_line_width=LINE_WIDTH,
                         text=player_stats_req['displayName'],
                         textposition='bottom center'),3,2)

fig.update_xaxes(title_text='Weight (lbs)',title_font_size=8,row=2,col=1)
fig.update_xaxes(title_text='Weight (lbs)',title_font_size=8,row=2,col=2)
fig.update_xaxes(title_text='Speed (yards/s)',title_font_size=8,row=3,col=1)
fig.update_xaxes(title_text='Height (inches)',title_font_size=8,row=3,col=2)

fig.update_yaxes(title_text='Speed (yards/s)',title_font_size=8,row=2,col=1)
fig.update_yaxes(title_text='Acceleration (yards/s<sup>2</sup>)',title_font_size=8,side='right',row=2,col=2)
fig.update_yaxes(title_text='Acceleration (yards/s<sup>2</sup>)',title_font_size=8,row=3,col=1)
fig.update_yaxes(title_text='Total Distance (yards)',side='right',title_font_size=8,row=3,col=2)

fig.update_layout(template='plotly_dark',width=700,height=800,
                  title='Player Statistics, 2018 Season',
                  title_font_family='Rockwell',margin=dict(l=20,r=20,t=50),
                  legend=dict(orientation='h',y=-0.1,font_size=8)
                 )
fig.show()                   

In [ ]:
game_player_details = pd.merge(player_game_stats,games,on='gameId')
game_player_details = pd.merge(game_player_details,players,on='nflId')
game_player_details.sort_values(['gameDateT','hours'],inplace=True)
game_player_details.reset_index(drop=True, inplace=True)
game_player_details['game'] = game_player_details.groupby('gameId').ngroup() + 1

gameplay_stats = game_player_details.groupby(['gameId','game','gameDateT','gameTimeEastern','pos_desc','type'],as_index=False)\
                   .agg(average_speed = ('s','mean'),
                        average_acc = ('a','mean'),
                        total_distance = ('dis','sum'))

In [ ]:
fig = px.bar(data_frame=gameplay_stats, x='game', y='total_distance', color='pos_desc',facet_row='type')
fig.update_traces(hovertemplate='%{y} yards')
fig.layout.xaxis.update(title_text='Game Number')
fig.layout.yaxis.update(matches=None,title_text='')
fig.layout.yaxis2.update(matches=None,title_text='Total Distance covered in yards')
fig.layout.yaxis3.update(matches=None,title_text='')
fig.for_each_annotation(lambda annot : annot.update(text=annot.text[5:]))
fig.update_layout(title = 'Total Distance covered per Game, 2018 Season', title_font_family='Rockwell',
                  template='plotly_dark', hovermode='x', barmode='stack',width=700,height=400,
                  margin=dict(l=10,r=10,t=50,b=10),
                  legend_title='Position', legend_font_size=7, legend_font_family='Rockwell')
fig.show()

In [ ]:
fig = px.scatter(data_frame=gameplay_stats, x='game', y='average_speed',
                 opacity=0.7,color='pos_desc',facet_row='type')
fig.update_traces(hovertemplate='%{y:.2f} yards/s')
fig.layout.xaxis.update(title_text='Game Number')
fig.layout.yaxis.update(matches=None,title_text='')
fig.layout.yaxis2.update(matches=None,title_text='Average Speed in yards/s')
fig.layout.yaxis3.update(matches=None,title_text='')
fig.for_each_annotation(lambda annot : annot.update(text=annot.text[5:]))
fig.update_layout(title = 'Average Speed per Game, 2018 Season', title_font_family='Rockwell',
                  template='plotly_dark', hovermode='x',width=700,height=400,
                  margin=dict(l=10,r=10,t=50,b=10),
                  legend_title='Position', legend_font_size=7, legend_font_family='Rockwell')
fig.show()

In [ ]:
fig = px.scatter(data_frame=gameplay_stats, x='game', y='average_acc',
                 opacity=0.7,color='pos_desc',facet_row='type')
fig.update_traces(hovertemplate='%{y:.2f} yards/s<sup>2</sup>')
fig.layout.xaxis.update(title_text='Game Number')
fig.layout.yaxis.update(matches=None,title_text='')
fig.layout.yaxis2.update(matches=None,title_text='Average Acceleration in yards/s<sup>2</sup>')
fig.layout.yaxis3.update(matches=None,title_text='')
fig.for_each_annotation(lambda annot : annot.update(text=annot.text[5:]))
fig.update_layout(title = 'Average Acceleration per Game, 2018 Season', title_font_family='Rockwell',
                  template='plotly_dark', hovermode='x',width=700,height=400,
                  margin=dict(l=10,r=10,t=50,b=10),
                  legend_title='Position', legend_font_size=7, legend_font_family='Rockwell')
fig.show()

# 🏈 Plays
In this section I have tried to visualize the plays. Below is an example of a random play.

In [ ]:
#Buffer to keep a week's data handy(Saves some time!)
buffer = pd.DataFrame()
if len(buffer) == 0:
    buffer = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2021/week1.csv')
    buffer.name = 'week1'

def add_newline(s):
    words = s.split()
    l = 0
    for i,word in enumerate(words):
        l += len(word) + 1
        if l < 80:
            continue
        else:
            words.insert(i, '<br>')
            l = 0
    return ' '.join(words)

def plot_gameplay(gameId, playId):
    """Presents the gameplay using game and play Id."""
    
    global buffer
    path = '/kaggle/input/nfl-big-data-bowl-2021/'
    
    #Get the title of the plot
    week,home,visitor,date,time = games.loc[games['gameId']==gameId,['week','homeTeamAbbr',
                                              'visitorTeamAbbr','gameDateT',
                                              'gameTimeEastern']].iloc[0]
    desc = add_newline(plays.loc[(plays['gameId']==gameId)&(plays['playId']==playId),
                     'playDescription'].iloc[0])
    title = f"<b>{home}(Home) vs {visitor}(Away), {time[:-3]} ET, {date.day} {date.month_name()}</b><br>{desc}"
    
    #Check if buffer can be used
    name = f"week{week}"
    if buffer.name != name:
        buffer = pd.read_csv(os.path.join(path,f"{name}.csv"))
        buffer.name = name
    game_data = buffer[(buffer['gameId']==gameId)&(buffer['playId']==playId)]
    
    #Plot the gameplay
    fig = px.scatter(data_frame=game_data,x='x',y='y',color='team',animation_frame='frameId',
                custom_data=['displayName','jerseyNumber','position','team'])
    fig.update_traces(marker_line_color='black', marker_line_width=1, marker_size=10)
    hovertemplate='<b>%{customdata[0]}</b><br><br>'+\
                  '<b>Jersey No.</b>: %{customdata[1]}<br>'+\
                  '<b>Position</b>: %{customdata[2]}<br>'+\
                  '<b>Team</b>: %{customdata[3]}<extra></extra>'
    fig.for_each_trace(lambda trace: trace.update(hovertemplate=None) if 'football' in trace.hovertemplate \
                       else trace.update(hovertemplate=hovertemplate))
    add_image_plotly(fig,'/kaggle/input/nfl2021images/Field.png',x=0,y=1,
                     opacity=1,sizex=1,sizey=1,sizing='stretch')
    
    fig.update_layout(title=title,showlegend=False, hovermode='closest',
                      width=700, height=500,margin=dict(l=0, r=0,t=150,b=0),
                      xaxis=dict(title='',range=[-3,123],showgrid=False,
                                 zeroline=False,showticklabels=False),
                      yaxis=dict(title='',range=[-3,56.3],showgrid=False,
                                 zeroline=False,showticklabels=False))
    fig.show()

In [ ]:
plot_gameplay(2018090600,75)

# Lot more analysis coming up. Stay Tuned :)